보스턴 데이터셋의 특징 출력

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install pandas

In [ ]:
!pip install tqdm

In [ ]:
from sklearn.datasets import fetch_california_housing

# 데이터셋 로드
housing = fetch_california_housing()

# 데이터 및 타겟 출력
print(housing.data.shape)
print(housing.target.shape)

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

(20640, 8)
(20640,)


# 데이터의 구성요소 확인

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing

# 캘리포니아 주택 데이터셋 로드
dataset = fetch_california_housing()

# 데이터셋의 데이터를 데이터프레임으로 변환
dataFrame = pd.DataFrame(dataset["data"], columns=dataset["feature_names"])

# 데이터 프레임에 타겟(정답) 추가
dataFrame["target"] = dataset["target"]

# 데이터프레임을 요약해서 출력
print(dataFrame.head())


   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  target  
0    -122.23   4.526  
1    -122.22   3.585  
2    -122.24   3.521  
3    -122.25   3.413  
4    -122.25   3.422  


# 선형회귀를 위한 MLP모델의 설계

In [ ]:
import torch
import torch.nn as nn

from torch.optim.adam import Adam


# ❶ 모델 정의
model = nn.Sequential(
   nn.Linear(8, 100),
   nn.ReLU(),
   nn.Linear(100, 1)
)

X = dataFrame.iloc[:, :-1].values # ❷ 정답을 제외한 특징을 X에 입력
Y = dataFrame["target"].values    # 데이터프레임의 target의 값을 추출

batch_size = 100
learning_rate = 0.001

# ❸ 가중치를 수정하기 위한 최적화 정의
optim = Adam(model.parameters(), lr=learning_rate)


# 에포크 반복
for epoch in range(200):

   # 배치 반복
   for i in range(len(X)//batch_size):
       start = i*batch_size      # ➍ 배치 크기에 맞게 인덱스를 지정
       end = start + batch_size


       # 파이토치 실수형 텐서로 변환
       x = torch.FloatTensor(X[start:end])
       y = torch.FloatTensor(Y[start:end])

       optim.zero_grad() # ❺ 가중치의 기울기를 0으로 초기화
       preds = model(x)  # ❻ 모델의 예측값 계산
       loss = nn.MSELoss()(preds, y) # ❼ MSE 손실 계산
       loss.backward()
       optim.step()

   if epoch % 20 == 0:
       print(f"epoch{epoch} loss:{loss.item()}")

epoch0 loss:5.053299427032471
epoch20 loss:0.8689953088760376
epoch40 loss:0.8201797604560852
epoch60 loss:1.1852997541427612
epoch80 loss:2.7027716636657715
epoch100 loss:5.048101425170898
epoch120 loss:4.8703413009643555
epoch140 loss:28.24327850341797
epoch160 loss:16.663436889648438
epoch180 loss:8.785953521728516


# 모델 성능 평가

In [ ]:
prediction = model(torch.FloatTensor(X[0, :13]))
real = Y[0]
print(f"prediction:{prediction.item()} real:{real}")

prediction:3.0476436614990234 real:4.526
